In [1]:
import csv
import io
import os
import pandas as pd
import requests
import sys
import xlwings as xw

from PyQt5.QtGui import QIcon
from PyQt5.QtCore import QDate, pyqtSlot
from PyQt5.QtWidgets import (QAction, QApplication, QCalendarWidget, QDateEdit, QGridLayout, QGroupBox, QHBoxLayout, QLabel, QLayout, QMainWindow, QWidget, QPushButton, QLineEdit, QTextEdit, QFileDialog)


class Window(QWidget):
    def __init__(self):
        super(Window, self).__init__()
        self.setWindowTitle("Query Cocorahs")
        self.initUI()


    def initUI(self):
        self.createPreviewGroupBox()
        self.createDatesGroupBox()
        self.infoPanel()

        layout = QGridLayout()
        # layout.addWidget(self.previewGroupBox, 0, 0)
        layout.addWidget(self.datesGroupBox, 0, 1)
        layout.addWidget(self.infoPanelBox, 0, 2, 0, 1)
        layout.setSizeConstraint(QLayout.SetFixedSize)
        self.setLayout(layout)
        self.previewLayout.setRowMinimumHeight(0,
                self.calendar.sizeHint().height())
        self.previewLayout.setColumnMinimumWidth(0,
                self.calendar.sizeHint().width())


    def createPreviewGroupBox(self):
        self.previewGroupBox = QGroupBox("Calendar")
        self.calendar = QCalendarWidget()
        self.calendar.setMaximumDate(QDate(3000, 1, 1))
        self.calendar.setGridVisible(True)
        self.previewLayout = QGridLayout()
        self.previewLayout.addWidget(self.calendar, 0, 0)
        self.previewGroupBox.setLayout(self.previewLayout)


    def createDatesGroupBox(self):
        self.datesGroupBox = QGroupBox(self.tr("Selections"))

        # QUERY
        self.queryButton = QPushButton("Query Database")
        self.queryButton.clicked.connect(self.query_on_click)

        # START DATES
        self.currentDateEdit = QDateEdit()
        self.currentDateEdit.setDisplayFormat('dd MMM yyyy')
        self.currentDateEdit.setDate(self.calendar.selectedDate())
        self.currentDateEdit.setDateRange(self.calendar.minimumDate(),
                self.calendar.maximumDate())
        self.currentDateLabel = QLabel("&Start Date:")
        self.currentDateLabel.setBuddy(self.currentDateEdit)

        # END DATES
        self.maximumDateEdit = QDateEdit()
        self.maximumDateEdit.setDisplayFormat('dd MMM yyyy')
        self.maximumDateEdit.setDate(self.calendar.selectedDate())
        self.maximumDateEdit.setDateRange(self.calendar.minimumDate(),
                self.calendar.maximumDate())
        self.maximumDateLabel = QLabel("&End Date:")
        self.maximumDateLabel.setBuddy(self.maximumDateEdit)

        # LONGITUDE
        self.textboxLong = QLineEdit(self)
        self.textboxLongLabel = QLabel("Longitude:")

        # LATITUDE
        self.textboxLat = QLineEdit(self)
        self.textboxLatLabel = QLabel("Latitude:")

        # STATES
        self.textboxStates = QLineEdit(self)
        self.textboxStatesLabel = QLabel("States: 'CO,NE,WY'")

        dateBoxLayout = QGridLayout()
        dateBoxLayout.addWidget(self.currentDateLabel, 1, 0)
        dateBoxLayout.addWidget(self.currentDateEdit, 1, 1, 1, 2)
        dateBoxLayout.addWidget(self.maximumDateLabel, 2, 0)
        dateBoxLayout.addWidget(self.maximumDateEdit, 2, 1, 1, 2)
        dateBoxLayout.addWidget(self.textboxLongLabel, 3, 0)
        dateBoxLayout.addWidget(self.textboxLong, 3, 1, 1, 2)
        dateBoxLayout.addWidget(self.textboxLatLabel, 4, 0)
        dateBoxLayout.addWidget(self.textboxLat, 4, 1, 1, 2)
        dateBoxLayout.addWidget(self.textboxStatesLabel, 5, 0)
        dateBoxLayout.addWidget(self.textboxStates, 5, 1, 1, 2)
        dateBoxLayout.addWidget(self.queryButton, 6, 0,1,3)
        dateBoxLayout.setRowStretch(5, 1)
        self.datesGroupBox.setLayout(dateBoxLayout)


    def infoPanel(self):
        self.infoPanelBox = QGroupBox(self.tr("Info Console"))
        self.infoConsole = QTextEdit(self)
        infoPanelBoxLayout = QGridLayout()
        infoPanelBoxLayout.addWidget(self.infoConsole, 0, 0, 1, 1)
        self.infoPanelBox.setLayout(infoPanelBoxLayout)



    def query_on_click(self):
        long = self.textboxLong.text()
        lat = self.textboxLat.text()
        startDate = self.currentDateEdit.date()
        endDate = self.maximumDateEdit.date()
        states = self.textboxStates.text()
        date_range = list(range(startDate.month(), endDate.month() + 1))

        # options = QFileDialog.Options()
        # options |= QFileDialog.DontUseNativeDialog
        # fileName, _ = QFileDialog.getOpenFileName(self,"Choose a database (.csv) to query", "","All Files (*);;Python Files (*.csv)", options=options)
        # if fileName:
        #     df = pd.read_csv(fileName)
        query(long=long, lat=lat, date_range=date_range, states=states)


def query(long, lat, date_range, states):
    print("hello")
    CSV_URL = 'http://data.cocorahs.org/export/exportreports.aspx?ReportType=hail&Format=csv&State=CO&ReportDateType=reportdate&Date=8/14/2018'

    with requests.Session() as s:
        data = s.get(CSV_URL)


    df = pd.read_csv(io.StringIO(data.text))
    print(df)



def download_file(url, filename):
    ''' Downloads file from the url and save it as filename '''
    # check if file already exists
    if not os.path.isfile(filename):
        print('Downloading File')
        response = requests.get(url)
        # Check if the response is ok (200)
        if response.status_code == 200:
            # Open file and write the content
            with open(filename, 'wb') as file:
                # A chunk of 128 bytes
                for chunk in response:
                    file.write(chunk)
    else:
        print('File exists')


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = Window()
    window.show()
    sys.exit(app.exec_())


hello
   ObservationDate ObservationTime         EntryDateTime StationNumber  \
0       2018-08-14        05:25 PM   2018-08-14 05:43 PM     CO-JF-157   
1       2018-08-14        05:10 PM   2018-08-14 05:19 PM     CO-BO-143   
2       2018-08-14        05:15 PM   2018-08-15 07:49 AM     CO-BO-149   
3       2018-08-14        05:04 PM   2018-08-14 05:13 PM      CO-PK-69   
4       2018-08-14        06:18 PM   2018-08-14 06:23 PM     CO-JF-157   
5       2018-08-14        05:12 PM   2018-08-14 05:18 PM     CO-WE-311   
6       2018-08-14        06:40 PM   2018-08-15 10:21 AM     CO-JF-240   
7       2018-08-14        05:58 PM   2018-08-14 07:11 PM     CO-AD-120   
8       2018-08-14        06:05 PM   2018-08-14 06:30 PM      CO-AD-87   
9       2018-08-14        05:51 PM   2018-08-14 06:14 PM     CO-DN-141   
10      2018-08-14        05:25 PM   2018-08-15 07:18 AM     CO-WE-302   
11      2018-08-14        12:00 AM   2018-08-15 07:40 AM     CO-JF-267   
12      2018-08-14        05:47 

SystemExit: 0

c:\users\dhaggerty\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import math
stationLatitude, stationLongitude = 39.579500, -105.278900
targetLatitude, targetLongitude = 40.200100, -105.108700
def distance(stationLatitude, stationLongitude, targetLatitude, targetLongitude):
    lat1, lon1 = stationLatitude, stationLongitude
    lat2, lon2 = targetLatitude, targetLongitude
    km_radius = 6371 # km
    mile_radius = 3959 # radius of the great circle in miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = mile_radius * c

    return d

print(distance(stationLatitude, stationLongitude, targetLatitude, targetLongitude))

43.82104525195731


In [4]:
import folium
latlon = [ (51.249443914705175, -0.13878830247011467), (51.249443914705175, -0.13878830247011467), (51.249768239976866, -2.8610415615063034)]
mapit = folium.Map( location=[52.667989, -1.464582], zoom_start=6 )
for coord in latlon:
    folium.Marker( location=[ coord[0], coord[1] ]).add_to( mapit )

mapit.save( 'map.html')

In [5]:
colors = {'A' : 'red', 'B' : 'blue'}

map_osm = folium.Map(location=[40.742, -73.956], zoom_start=11)

df.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], 
                                              radius=10, fill_color=red)
                                             .add_to(map_osm), axis=1)

map_osm

NameError: name 'df' is not defined